<a href="https://colab.research.google.com/github/ymohamedahmed/drbayes/blob/master/SIBDL_demo_group_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning Seminars - Subspace Inference for Bayesian Deep Learning - Demo

*Reviewed by Chiara Campagnola, Yousuf Mohamed-Ahmed and Hannah Teufel*

**Aims of this notebook**:
- Empirically evaluate the effectiveness of the uncertainty estimates produced (this is not evaluated by the paper)
- Evaluate the performance gains achieved by applying approximate inference techniques in subspaces *vs* in the full space
- Compare the method proposed by the paper to non-neural network based approaches

In [1]:
!rm -rf drbayes
!git clone https://github.com/ymohamedahmed/drbayes.git

Cloning into 'drbayes'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 349 (delta 43), reused 51 (delta 22), pack-reused 261
Receiving objects: 100% (349/349), 11.61 MiB | 25.68 MiB/s, done.
Resolving deltas: 100% (113/113), done.


In [2]:
!pip install -e drbayes

Obtaining file:///content/drbayes
     |████████████████████████████████| 286kB 11.0MB/s 
     |████████████████████████████████| 614kB 18.3MB/s 
  Created wheel for gpytorch: filename=gpytorch-1.4.0-py2.py3-none-any.whl size=477826 sha256=6a6d71d86d49ffe35bbca58b2bb017bda4bd47728af959513799b5d8faa7ad1e
  Stored in directory: /root/.cache/pip/wheels/fd/f5/39/404e1875f841e8a999e94a7efa17f6ef900298be5452b63b0c
Successfully built gpytorch
  Running setup.py develop for subspace-inference


In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.utils.data
from torch.nn import functional as F
import torch.nn as nn
from torchvision import datasets, transforms

import seaborn as sns

import subspace_inference
import subspace_inference.utils as utils
from subspace_inference.posteriors import SWAG
from subspace_inference import models, losses, utils
from subspace_inference.models import MLP
from subspace_inference.visualization import plot_predictive
from subspace_inference.posteriors.proj_model import SubspaceModel
from tqdm import tqdm

import os

torch.backends.cudnn.benchmark = True
torch.manual_seed(1)
torch.cuda.manual_seed(1)
np.random.seed(1)

%load_ext autoreload
%autoreload 2

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [13]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
train = datasets.QMNIST(root="../data",train=True, download=True,
                   transform=transform)
test = datasets.QMNIST('../data', train=False,download=True,
                   transform=transform)
train_loader = torch.utils.data.DataLoader(train,batch_size=512)
test_loader = torch.utils.data.DataLoader(test,batch_size=512)

## Models

In [ ]:
def train(model, loader, optimizer, criterion, lr_init=1e-2, epochs=3000, 
          swag_model=None, swag=False, swag_start=2000, swag_freq=50, swag_lr=1e-3,
          print_freq=100):
    
    for epoch in range(epochs):
        t = (epoch + 1) / swag_start if swag else (epoch + 1) / epochs
        lr_ratio = swag_lr / lr_init if swag else 0.05
        
        if t <= 0.5:
            factor = 1.0
        elif t <= 0.9:
            factor = 1.0 - (1.0 - lr_ratio) * (t - 0.5) / 0.4
        else:
            factor = lr_ratio

        lr = factor * lr_init
        utils.adjust_learning_rate(optimizer, lr)
        
        train_res = utils.train_epoch(loader, model, criterion, optimizer, cuda=False, regression=False)
        if swag and epoch > swag_start:
            swag_model.collect_model(model)
        
        if (epoch % print_freq == 0 or epoch == epochs - 1):
            print('Epoch %d. LR: %g. Loss: %.4f' % (epoch, lr, train_res['loss']))


In [ ]:
wd = 0.
lr_init = 1e-2

model_cfg = models.ToyRegNet
criterion = losses.GaussianLikelihood(noise_var=1.)
criterion = F.cross_entropy
model_cfg.kwargs = {"dimensions":[20,20], "output_dim":10, "input_dim":28*28}
model = model_cfg.base(*model_cfg.args, **model_cfg.kwargs)
for i in range(2):
    print("Training Model", i)
    swag_model = SWAG(model_cfg.base, subspace_type="pca", *model_cfg.args, **model_cfg.kwargs, 
                  subspace_kwargs={"max_rank": 10, "pca_rank": 10})
    model = model_cfg.base(*model_cfg.args, **model_cfg.kwargs)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr_init, momentum=0.95, weight_decay=wd)
    
    train(model, train_loader, optimizer, criterion, lr_init, 3000, print_freq=1000, 
          swag=True, swag_model=swag_model, swag_start=2000, swag_freq=10, swag_lr=1e-2)

Training Model 0


ModuleAttributeError: ignored

In [11]:

def pretrain_mlp(model, loss_function, max_epochs, train_loader):
  optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
  swag_model = SWAG(VanillaMLP, subspace_type="pca",
                  subspace_kwargs={"max_rank": 10, "pca_rank": 10},dims=[28*28,50,20,10])
  for epoch in range(max_epochs):
    total_loss = 0
    for x,y in train_loader:
      x = x.to(device)
      y = y.to(device)
      optimizer.zero_grad()
      out = model(x)
      swag_model.collect_model(model)

      loss = loss_function(out,y) 
      #if not(elbo) else loss_function(model,x,y)[0]
      # total_loss += loss
      loss.backward()
      optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, loss: {loss}")

  return swag_model.get_space()

def train_vi_model(model, loss_function, max_epochs, train_loader):
  optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
  for epoch in range(max_epochs):
    total_loss = 0
    for x,y in train_loader:
      x = x.to(device)
      y = y.to(device)
      optimizer.zero_grad()
      out = model(x)
      loss = loss_function(model,x,y)[0]
      loss.backward()
      optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, loss: {loss}")


### Mean Field Variational Inference

### Ensembles

### Neural network + Bayesian Linear Regression

### _SIBDL_: PCA subspace

In [6]:
class VanillaMLP(nn.Module):
  def __init__(self, dims):
    super(VanillaMLP,self).__init__()
    layers = [nn.Flatten()] + [lay for (x,y) in zip(dims[:-1],dims[1:]) for lay in [nn.Linear(x,y), nn.ReLU()] ]
    layers.pop()
    
    self.model = nn.Sequential(*layers)

  def forward(self,x):
    return self.model(x)

In [ ]:
model = VanillaMLP([28*28,50,20,10])
print(model)

VanillaMLP(
  (model): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=50, bias=True)
    (2): ReLU()
    (3): Linear(in_features=50, out_features=20, bias=True)
    (4): ReLU()
    (5): Linear(in_features=20, out_features=10, bias=True)
  )
)


In [9]:
model = VanillaMLP([28*28,50,20,10])
model.to(device)
space = pretrain_mlp(model, F.cross_entropy, 20, train_loader)
torch.save(model.state_dict(), "MLP.pt")

Epoch: 0, loss: 0.3785310983657837
Epoch: 10, loss: 0.09515810012817383


In [12]:
from subspace_inference.posteriors.vi_model import VIModel, ELBO
import math
def get_pca_space(space):
    # swag_model = SWAG(model_cfg.base, subspace_type="pca", *model_cfg.args, **model_cfg.kwargs, 
    #               subspace_kwargs={"max_rank": 10, "pca_rank": 10})
    # print(torch.load("MLP.pt").keys())
    # swag_model.load_state_dict(torch.load("MLP.pt"))#["state_dict"])
    # mean, _, cov_factor = swag_model.get_space()
    mean, _, cov_factor = space
    subspace = SubspaceModel(mean, cov_factor)
    return subspace

subspace = get_pca_space(space)
init_sigma = 1.
prior_sigma = 5.
criterion = losses.GaussianLikelihood(noise_var=.05)
criterion = losses.cross_entropy
temperature = 1.

vi_model = VIModel(
    subspace=subspace,
    init_inv_softplus_sigma=math.log(math.exp(init_sigma) - 1.0),
    prior_log_sigma=math.log(prior_sigma),
    base=VanillaMLP,
    dims=[28*28,50,20,10]
)

elbo = ELBO(criterion, len(train_loader.dataset), temperature=temperature)
vi_model.to(device)
train_vi_model(vi_model, elbo, 30, train_loader)

Epoch: 0, loss: 0.14315642416477203
Epoch: 10, loss: 0.01816697046160698
Epoch: 20, loss: 0.041046857833862305


In [72]:
from tqdm.notebook import tqdm
num_evals = 1000
test_batch_size = 512
preds = torch.zeros((len(test_loader.dataset), num_evals))
print(len(test_loader.dataset))
for n, (x,y) in tqdm(enumerate(test_loader)):
  x = x.to(device)
  for i in range(num_evals):
    preds[n*test_batch_size:(n+1)*test_batch_size,i] = vi_model(x).argmax(axis=1)

60000


In [74]:
uncert = torch.std(torch.tensor(~torch.eq(preds.T, torch.mode(preds,dim=1)[0]),dtype=torch.float32),dim=0)
acc = (1.0*torch.mode(preds,dim=1)[0].eq(test_loader.dataset.targets[:,0])).mean()
acc

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


tensor(0.9660)

In [1]:
inds = (-1.*uncert).argsort()[:10]
print(uncert[inds])
print(inds)
for im in test_loader.dataset.data[inds]:
  plt.imshow(im, cmap='gray')
  plt.show()

NameError: ignored

## _References_

- A very useful repository for a lot of Bayesian NN implementations: https://github.com/JavierAntoran/Bayesian-Neural-Networks
- The code for the paper is found at https://github.com/wjmaddox/drbayes and specifically the following notebook was adapted for this demonstration (https://github.com/wjmaddox/drbayes/blob/master/experiments/synthetic_regression/visualizing_uncertainty.ipynb)
